In [116]:
import pandas as pd
import numpy as np
import pycaret.classification as pc

#import matplotlib.pyplot as plt

import matplotlib

import mlflow
from mlflow.models.signature import infer_signature
from mlflow.client import MlflowClient


# Configurar MLFlow

In [109]:
# Para usar o sqlite como repositorio
mlflow.set_tracking_uri("sqlite:///mlruns.db")

experiment_name = 'Projeto Kobe'
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
    experiment = mlflow.get_experiment(experiment_id)
experiment_id = experiment.experiment_id

In [110]:
from sklearn.model_selection import train_test_split

data_cols = ['lat','lon','minutes_remaining','period','playoffs','shot_distance','shot_made_flag']
train_perc = 0.8

with mlflow.start_run(experiment_id=experiment_id, run_name = 'PreparacaoDados'):

  df_dev = pd.read_parquet('../data/raw/dataset_kobe_dev.parquet')
  df_dev = df_dev[data_cols].copy()

  df_dev.to_parquet("../data/processed/data_filtered.parquet")
  df_dev = df_dev.dropna(subset=['shot_made_flag'])
  xtrain, xtest, ytrain, ytest = train_test_split(df_dev[['lat','lon','minutes_remaining','period','playoffs','shot_distance']],
                                                  df_dev['shot_made_flag'],
                                                  train_size=train_perc,
                                                  stratify=df_dev['shot_made_flag'])

  xtrain['shot_made_flag'] = ytrain
  xtest['shot_made_flag'] = ytest
  
  xtrain.to_parquet('../data/processed/base_train.parquet')
  xtest.to_parquet('../data/processed/base_test.parquet')

  mlflow.log_params({
      'perc_test': 1-train_perc,
      'colunas-selecionadas': data_cols
      })
  mlflow.log_metrics({
      'qtd_linhas_treino': xtrain.shape[0],
      'qtd_linhas_teste': xtest.shape[0],
  })

# Treinamento do Modelo

In [111]:
exp = pc.setup(
    data=xtrain,
    target = 'shot_made_flag',
    test_data = xtest,
    normalize=True,
    log_experiment = False
)
list_models = exp.compare_models(['lr','dt'], n_select=2, sort='f1')
list_models
exp.plot_model(list_models[0], plot='vc', save = True)

,Description,Value
0,Session id,8688
1,Target,shot_made_flag
2,Target type,Binary
3,Original data shape,"(20285, 7)"
4,Transformed data shape,"(20285, 7)"
5,Transformed train set shape,"(16228, 7)"
6,Transformed test set shape,"(4057, 7)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,0.5320,0.5119,0.5750,0.5088,0.5398,0.0673,0.0678,0.0180
lr,Logistic Regression,0.5784,0.6013,0.4893,0.5677,0.5255,0.1499,0.1513,0.2420


'Validation Curve.png'

In [112]:
from sklearn.model_selection import validation_curve

def plot_parameter_validation_curve(X, Y, param_name, grid_search,
                                    model, model_name, scoring,
                                    logx):
    print('Parameter:', param_name)
    print('GridSearch:', grid_search[param_name])
    print('Scoring:', scoring)
    plt.figure(figsize=(6,4))
    train_scores, test_scores = validation_curve(model,
                                                 X = X,
                                                 y = Y,
                                                 param_name=param_name,
                                                 param_range= grid_search[param_name],
                                                 scoring=scoring,
                                                 cv=10,
                                                 n_jobs=-1)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.title("Curva Validação Modelo " + model_name)
    plt.xlabel(param_name)
    plt.ylabel("Score ("+scoring+")")
    if logx:
        plt.semilogx(grid_search[param_name], train_scores_mean,'-o', label="Treino",
                     color="darkorange", lw=2)
        plt.semilogx(grid_search[param_name], test_scores_mean,'-o', label="Validação-Cruzada",
                     color="navy", lw=2)
    else:
        plt.plot(grid_search[param_name], train_scores_mean,'-o', label="Treino",
                     color="darkorange", lw=2)
        plt.plot(grid_search[param_name], test_scores_mean,'-o', label="Validação-Cruzada",
                 color="navy", lw=2)
    plt.fill_between(grid_search[param_name], train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.2,
                     color="darkorange", lw=2)
    plt.fill_between(grid_search[param_name], test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.2,
                     color="navy", lw=2)
    plt.legend(loc='best')
    plt.grid(True)
    return plt.gcf()

In [113]:
mlflow.end_run()

In [114]:
import os
from sklearn.metrics import log_loss, f1_score

registered_model_name = 'modelo_kobe'
nexamples = 5
model_version = -1

with mlflow.start_run(experiment_id=experiment_id, run_name = 'Treinamento'):

    exp = pc.setup(
        data=xtrain,
        target = 'shot_made_flag',
        test_data = xtest,
        normalize=True,
        log_experiment = False
    )
    list_models = exp.compare_models(['lr','dt'], n_select=2, sort='f1')
    
    # REGRESSÃO LOGÍSTICA
    yhat_test = exp.predict_model(list_models[1])
    plot_parameter_validation_curve(xtrain.drop('shot_made_flag', axis=1), ytrain, 'C', {'C': [0.001, 0.01, 0.1, 1, 10]}, list_models[1], 'Regressão Logística', 'f1', logx=True)
    
    mlflow.log_metrics({
        'lr_log_loss': log_loss(yhat_test.shot_made_flag, yhat_test.prediction_label),
        'lr_f1': f1_score(yhat_test.shot_made_flag, yhat_test.prediction_label)
    })

    plt.savefig('lr_validation_curve.png')
    mlflow.log_artifact('lr_validation_curve.png')
    # os.remove('lr_validation_curve.png')

    # ÁRVORE DECISÃO
    yhat_test = exp.predict_model(list_models[0])
    plot_parameter_validation_curve(xtrain.drop('shot_made_flag', axis=1), ytrain, 'max_depth', {'max_depth': [2, 3, 4, 5, 6, 7, 8]}, list_models[0], 'Árvore Decisão', 'f1', logx=False)

    mlflow.log_metrics({
        'dt_log_loss': log_loss(yhat_test.shot_made_flag, yhat_test.prediction_label),
        'dt_f1': f1_score(yhat_test.shot_made_flag, yhat_test.prediction_label)
    })
    plt.savefig('dt_validation_curve.png')
    mlflow.log_artifact('dt_validation_curve.png')
    # os.remove('dt_validation_curve.png')

    # FINALIZAÇÃO MELHOR MODELO
    tune_model = exp.tune_model(list_models[1],
                                optimize='f1',
                                search_library='scikit-learn',
                                search_algorithm='random',
                                n_iter = 4)

    yhat_test = exp.predict_model(tune_model)
    mlflow.log_metrics({
        'final_model_log_loss': log_loss(yhat_test.shot_made_flag, yhat_test.prediction_label),
        'final_model_f1': f1_score(yhat_test.shot_made_flag, yhat_test.prediction_label)
    })
    final_model = exp.finalize_model(tune_model)

    # EXPORTAÇÃO PARA LOG E REGISTRO DO MODELO
    exp.save_model(final_model, f'./{registered_model_name}') 
    # Carrega novamente o pipeline + bestmodel
    model_pipe = exp.load_model(f'./{registered_model_name}')
    # Assinatura do Modelo Inferida pelo MLFlow
    model_features = list(xtrain.drop('shot_made_flag', axis=1).columns)
    inf_signature = infer_signature(xtrain[model_features],
                                    model_pipe.predict(xtrain.drop('shot_made_flag', axis=1)))
    # Exemplo de entrada para o MLmodel
    input_example = {x: xtrain[x].values[:nexamples] for x in model_features}
    # Log do pipeline de modelagem do sklearn e registrar como uma nova versao
    mlflow.sklearn.log_model(
        sk_model=model_pipe,
        artifact_path="sklearn-model",
        registered_model_name=registered_model_name,
        signature = inf_signature,
        input_example = input_example
    )
    # Criacao do cliente do servico MLFlow e atualizacao versao modelo
    client = MlflowClient()
    if model_version == -1:
        model_version = client.get_latest_versions(registered_model_name)[-1].version
    # Registrar o modelo como staging
    client.set_registered_model_alias(
        name=registered_model_name,
        alias='staging',
        version=model_version
    )
    

,Description,Value
0,Session id,2449
1,Target,shot_made_flag
2,Target type,Binary
3,Original data shape,"(20285, 7)"
4,Transformed data shape,"(20285, 7)"
5,Transformed train set shape,"(16228, 7)"
6,Transformed test set shape,"(4057, 7)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,0.5319,0.5119,0.5736,0.5088,0.5391,0.0670,0.0674,0.0180
lr,Logistic Regression,0.5784,0.6013,0.4893,0.5677,0.5255,0.1499,0.1513,0.0130


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5723,0.5884,0.4827,0.5606,0.5187,0.1378,0.1390


Parameter: C
GridSearch: [0.001, 0.01, 0.1, 1, 10]
Scoring: f1


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Decision Tree Classifier,0.5201,0.5061,0.5617,0.4977,0.5278,0.0435,0.0439


...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5730,0.6010,0.5497,0.5532,0.5515,0.1441,0.1441
1,0.5853,0.5978,0.5381,0.5697,0.5534,0.1670,0.1672
2,0.5669,0.5896,0.5239,0.5486,0.5360,0.1303,0.1304
3,0.6026,0.6269,0.5845,0.5838,0.5841,0.2036,0.2036
4,0.5687,0.5937,0.5187,0.5514,0.5346,0.1334,0.1337
5,0.5564,0.5918,0.5277,0.5360,0.5319,0.1104,0.1104
6,0.5718,0.6041,0.5556,0.5506,0.5531,0.1421,0.1421
7,0.5810,0.6075,0.5568,0.5612,0.5590,0.1600,0.1600
8,0.5814,0.6054,0.5233,0.5664,0.5440,0.1582,0.1586


Fitting 10 folds for each of 4 candidates, totalling 40 fits


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.5662,0.5884,0.5276,0.5474,0.5373,0.1292,0.1293


Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Loaded


Registered model 'modelo_kobe' already exists. Creating a new version of this model...
Created version '10' of model 'modelo_kobe'.


In [115]:
final_model

Pipeline(memory=FastMemory(location=/var/folders/4b/wnq_s9nd58gbs_twrl61kk5m0000gn/T/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['lat', 'lon', 'minutes_remaining',
                                             'period', 'playoffs',
                                             'shot_distance'],
                                    transformer=SimpleImputer(add_indicator=False,
                                                              copy=True,
                                                              fill_value=None,
                                                              keep_empty_features=False,
                                                              missing_values=nan,
                                                              st...
                                    transformer=StandardScaler(copy=True,
                                                               with_mean=True,
                                                               with_std=True))),
                ('actual_estimator',
                 LogisticRegression(C=2.028, class_weight='balanced',
                                    dual=False, fit_intercept=True,
                                    intercept_scaling=1, l1_ratio=None,
                                    max_iter=1000, multi_class='auto',
                                    n_jobs=None, penalty='l2',
                                    random_state=2449, solver='lbfgs',
                                    tol=0.0001, verbose=0, warm_start=False))],
         verbose=False)